Few ideas so far:

## 1 FFNN based
- Train an nn on-top of sentence transformer to output logits / probs for score
  - optimise loss function against given scores
- Maybe experiment with longformer or othe large context window model instead of base sentence-transformer models as we might not be able to contain the entirety of the original text in smaller / short-passage model. 
- Longformer might need fine-tuning on sentence-pair datasets 
  - rationale https://stackoverflow.com/questions/63461262/bert-sentence-embeddings-from-transformers/64237402#64237402
- Try qlora technique to fine-tune llm on the prompt structure

## 2 LLM based
- Other approach might be to get a good language model to summarise the given prompt,
- embed its output
- embed the inference input
- get the absolute difference between the two, using the llm output as a ground truth
  - the closer the distance, the better the score we assign to the prompt
- could also experiment with fine-tuning the llm against the train dataset, or other summarisation datasets
  - would be a good opportunity to experiment with techniques such as peft and lora


## 3 ML
- There are some correlations between wordcount and scoring values, so try to fit a standard regression model on these figures to output scores
- Experiment with some more feature engineering, 
  - amount of unique words / n-grams
  - amount of shared n-grams
  - amount of stopwords in summarisation
  - average + stddev sentence length in summarisation
  - count of named entities shared in both
  - etc ...

## 4 ensemble
- Ensemble everything, maybe using somehing like rank averaging

## 1 - Longformer experiments

In [1]:
from transformers import LongformerTokenizerFast, LongformerModel
import torch

longformer = 'allenai/longformer-base-4096'

tokenizer = LongformerTokenizerFast.from_pretrained(longformer)
model = LongformerModel.from_pretrained(longformer)

/home/chris/repositories/student-summary-evaluation/.env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/chris/repositories/student-summary-evaluation/.env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
sample_text = ' '.join(['hello world! '] * 1000)

encoding = tokenizer.encode_plus(sample_text)
input_ids = torch.tensor(encoding['input_ids']).unsqueeze(0)

attention_mask = torch.tensor(encoding['attention_mask']).unsqueeze(0)

global_attention_mask = torch.zeros(
    input_ids.shape, dtype=torch.long, device=input_ids.device
)

For our purposes, the `global_attention_mask` vectors needs to be set on the tokens contained in the question part of the input string,

so for example say the quesition starts on character 21 and end on character 99.

reference
- https://colab.research.google.com/github/patil-suraj/Notebooks/blob/master/longformer_qa_training.ipynb#scrollTo=tty8vuMBqI5L
- https://huggingface.co/docs/transformers/model_doc/longformer

In [3]:
start_idx, end_idx = 21, 99

# translate indices from string to token seq
question_start = encoding.char_to_token(start_idx)
question_end = encoding.char_to_token(end_idx)

# Attend to all question tokens
global_attention_mask[:, [range(question_start, question_end)]] = 1
global_attention_mask

tensor([[0, 0, 0,  ..., 0, 0, 0]])

In [4]:
outputs = model(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)

In [5]:
sequence_output = outputs.last_hidden_state
pooled_output = outputs.pooler_output

In [6]:
sequence_output.shape, pooled_output.shape

(torch.Size([1, 4002, 768]), torch.Size([1, 768]))

For the output / embedding vector, could maybe take the hidden-state relating to the sentence start before the answer